# Introduction to Convolutional Neural Networks

**Goal** : This notebook explains how to implement a simple convolutional neural network model. 

## MLP with Keras

In [1]:
import numpy as np
import pandas as pd
import os
from PIL import Image

### Load Data

In [2]:
#Load the data

labels = pd.read_csv("../data/BeeSpotter/train_labels.csv")
submissionFormat = pd.read_csv("../data/BeeSpotter/SubmissionFormat.csv")

In [3]:
labels.head()

,id,genus
0,520,1
1,3800,1
2,3289,1
3,2695,1
4,4922,1


In [4]:
submissionFormat.head()

,id,genus
0,2783,0.5
1,2175,0.5
2,4517,0.5
3,2831,0.5
4,3556,0.5


In [5]:
print submissionFormat.shape, labels.shape

(992, 2) (3969, 2)


##### Util to get an image. This will also help us resize the image after we read it

In [6]:
def get_image(filename, folder, size):
    filepath = os.path.join("../data/BeeSpotter/images", folder, filename + ".jpg")
    image = Image.open(filepath)
    image.thumbnail((size, size), Image.ANTIALIAS)
    return (np.array(image, dtype=np.int32)[:,:,:3]).swapaxes(0,2)

##### Util to load all the images. This will call the `get_image` function defined above

In [7]:
def load_images(filenames, folder, size):
    features = np.empty([len(filenames), 3, size, size])
    for counter, file in enumerate(filenames):
#        print file, counter
        features[counter] = get_image(str(file), folder, size)
    
    return features

##### Load the train and test datasets

In [8]:
train_X = load_images(labels.id, "train", 32)
test_X = load_images(submissionFormat.id, "test", 32)

In [9]:
print train_X.shape, test_X.shape

(3969, 3, 32, 32) (992, 3, 32, 32)


##### Load `keras` package for CNN

In [10]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD

Using gpu device 0: GeForce GT 650M


##### First Model: Just one convolution layer

In [11]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, 3, border_mode='full')) 

model.add(Flatten())

model.add(Dense(32*34*34, 1))
model.add(Activation('sigmoid'))

In [12]:
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)
model.fit(train_X, np.array(labels.genus), batch_size=32, nb_epoch=10)

Epoch 0
3969/3969 [==============================] - 3s - loss: 0.0000     
Epoch 1
3969/3969 [==============================] - 2s - loss: 0.0000     
Epoch 2
3969/3969 [==============================] - 2s - loss: 0.0000     
Epoch 3
3969/3969 [==============================] - 2s - loss: 0.0000     
Epoch 4
3969/3969 [==============================] - 2s - loss: 0.0000     
Epoch 5
3969/3969 [==============================] - 2s - loss: -0.0000     
Epoch 6
3969/3969 [==============================] - 2s - loss: 0.0000     
Epoch 7
3969/3969 [==============================] - 2s - loss: -0.0000     
Epoch 8
3969/3969 [==============================] - 2s - loss: -0.0000     
Epoch 9
3969/3969 [==============================] - 3s - loss: -0.0000     


##### Second Model: One convolution layer and one RELU activation

In [13]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, 3, border_mode='full')) 

model.add(Convolution2D(32, 32, 3, 3))
model.add(Activation('relu'))

model.add(Flatten())

model.add(Dense(32*32*32, 1))
model.add(Activation('sigmoid'))

In [14]:
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)
model.fit(train_X, np.array(labels.genus), batch_size=32, nb_epoch=10)

Epoch 0
3969/3969 [==============================] - 5s - loss: -0.0000     
Epoch 1
3969/3969 [==============================] - 4s - loss: -0.0000     
Epoch 2
3969/3969 [==============================] - 4s - loss: -0.0000     
Epoch 3
3969/3969 [==============================] - 4s - loss: -0.0000     
Epoch 4
3969/3969 [==============================] - 4s - loss: -0.0000     
Epoch 5
3969/3969 [==============================] - 4s - loss: -0.0000     
Epoch 6
3969/3969 [==============================] - 4s - loss: -0.0000     
Epoch 7
3969/3969 [==============================] - 4s - loss: -0.0000     
Epoch 8
3969/3969 [==============================] - 4s - loss: -0.0000     
Epoch 9
3969/3969 [==============================] - 4s - loss: -0.0000     


##### Third Model: One convolution layer, one RELU activation and one MaxPooling layer

In [16]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, 3, border_mode='full')) 

model.add(Convolution2D(32, 32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(poolsize=(2, 2)))

model.add(Flatten())

model.add(Dense(32*16*16, 1))
model.add(Activation('sigmoid'))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)
model.fit(train_X, np.array(labels.genus), batch_size=32, nb_epoch=10)

Epoch 0
3969/3969 [==============================] - 5s - loss: -0.0000     
Epoch 1
3969/3969 [==============================] - 4s - loss: -0.0000     
Epoch 2
3969/3969 [==============================] - 4s - loss: -0.0000     
Epoch 3
3969/3969 [==============================] - 4s - loss: -0.0000     
Epoch 4
3969/3969 [==============================] - 4s - loss: -0.0000     
Epoch 5
3969/3969 [==============================] - 4s - loss: -0.0000     
Epoch 6
3969/3969 [==============================] - 4s - loss: -0.0000     
Epoch 7
3969/3969 [==============================] - 4s - loss: -0.0000     
Epoch 8
3969/3969 [==============================] - 4s - loss: -0.0000     
Epoch 9
3969/3969 [==============================] - 4s - loss: -0.0000     


##### Fourth Model: One convolution layer, one RELU activation, one MaxPooling layer with dropout

In [18]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, 3, border_mode='full')) 

model.add(Convolution2D(32, 32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(poolsize=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(32*16*16, 1))
model.add(Activation('sigmoid'))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)
model.fit(train_X, np.array(labels.genus), batch_size=32, nb_epoch=10)

Epoch 0
3969/3969 [==============================] - 5s - loss: -0.0000     
Epoch 1
3969/3969 [==============================] - 4s - loss: -0.0000     
Epoch 2
3969/3969 [==============================] - 4s - loss: -0.0000     
Epoch 3
3969/3969 [==============================] - 5s - loss: -0.0000     
Epoch 4
3969/3969 [==============================] - 4s - loss: -0.0000     
Epoch 5
3969/3969 [==============================] - 4s - loss: -0.0000     
Epoch 6
3969/3969 [==============================] - 4s - loss: -0.0000     
Epoch 7
3969/3969 [==============================] - 4s - loss: -0.0000     
Epoch 8
3969/3969 [==============================] - 5s - loss: -0.0000     
Epoch 9
3969/3969 [==============================] - 4s - loss: -0.0000     


##### A deep CNN

In [19]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, 3, border_mode='full')) 
model.add(Activation('relu'))
model.add(Convolution2D(32, 32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(poolsize=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 32, 3, 3, border_mode='full')) 
model.add(Activation('relu'))
model.add(Convolution2D(64, 64, 3, 3)) 
model.add(Activation('relu'))
model.add(MaxPooling2D(poolsize=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64*8*8, 256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256, 1))
model.add(Activation('sigmoid'))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)
model.fit(train_X, np.array(labels.genus), batch_size=32, nb_epoch=10)

Epoch 0
3969/3969 [==============================] - 9s - loss: -0.0000     
Epoch 1
3969/3969 [==============================] - 9s - loss: -0.0000     
Epoch 2
3969/3969 [==============================] - 9s - loss: -0.0000     
Epoch 3
3969/3969 [==============================] - 9s - loss: -0.0000     
Epoch 4
3969/3969 [==============================] - 9s - loss: -0.0000     
Epoch 5
3969/3969 [==============================] - 9s - loss: -0.0000     
Epoch 6
3969/3969 [==============================] - 9s - loss: -0.0000     
Epoch 7
3969/3969 [==============================] - 9s - loss: -0.0000     
Epoch 8
3969/3969 [==============================] - 9s - loss: -0.0000     
Epoch 9
3969/3969 [==============================] - 9s - loss: -0.0000     
